<a href="https://colab.research.google.com/github/Cirediallo/Models/blob/main/icefall_asr_tedlium3_pretrained_transducer_stateless_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
This notebook shows how to use a pre-trained Transducer Stateless model with [icefall](https://github.com/k2-fsa/icefall/).

## Environment setup

To use a pre-trained model with icefall, we have to install the following dependencies:

- [k2][k2], for FSA operations
- [torchaudio][audio], for reading sound files
- [kaldifeat][kaldifeat], for extracting features from a single sound
  file or multiple sound files

**NOTE**: [lhotse][lhotse] is used only in training time, for data preparation.


[k2]: https://github.com/k2-fsa/k2
[audio]: https://github.com/pytorch/audio
[kaldifeat]: https://github.com/csukuangfj/kaldifeat
[lhotse]: https://github.com/lhotse-speech/lhotse

### Install PyTorch and torchaudio



In [ ]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [20]:
! pip uninstall -y torch torchaudio torchvision torchtext

! pip install torch==1.7.1+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torch 1.13.1
Uninstalling torch-1.13.1:
  Successfully uninstalled torch-1.13.1
Found existing installation: torchaudio 0.13.1
Uninstalling torchaudio-0.13.1:
  Successfully uninstalled torchaudio-0.13.1
Found existing installation: torchvision 0.15.1+cu118
Uninstalling torchvision-0.15.1+cu118:
  Successfully uninstalled torchvision-0.15.1+cu118
Found existing installation: torchtext 0.15.1
Uninstalling torchtext-0.15.1:
  Successfully uninstalled torchtext-0.15.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu101 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+

### Install k2

In [3]:
! pip install torch
! pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install k2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 

In [22]:
! pip install k2==1.10.dev20211102+cuda10.1.torch1.7.1 -f https://k2-fsa.org/nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://k2-fsa.org/nightly
ERROR: Could not find a version that satisfies the requirement k2==1.10.dev20211102+cuda10.1.torch1.7.1 (from versions: 0.0.1.dev0, 0.0.2.dev0, 0.0.3.dev20201019, 0.0.3.dev20201020, 0.0.3.dev20201025, 0.0.3.dev20201027, 0.0.4.dev20201027, 0.0.5.dev20201027, 0.0.5.dev20201028, 0.0.6.dev20201028, 0.0.7.dev20201028, 0.0.9.dev20201109, 0.0.10.dev20201110, 0.1, 0.1.1.dev20201116, 0.1.1.dev20201118, 0.1.1.dev20201130, 0.1.1.dev20201208, 0.1.1.dev20201210, 0.1.1.dev20201212, 0.1.1.dev20201216, 0.1.2.dev20210111, 0.1.2, 0.1.3.dev20210111, 0.3.0, 0.3.1.dev20210121, 0.3.1.dev20210127, 0.3.1.dev20210204, 0.3.2.dev20210204, 0.3.3.dev20210205, 0.3.3.dev20210206, 0.3.3.dev20210209, 0.3.3.dev20210218, 0.3.3.dev20210222, 0.3.3.dev20210224, 0.3.3.dev20210302, 0.3.3.dev20210305, 0.3.3.dev20210309, 0.3.3.dev20210321, 0.3.3.dev20210328, 0.3.3.dev20210331, 0.3.3.de

Check that k2 was installed successfully:

In [5]:
! python3 -m k2.version

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/k2/__init__.py", line 23, in <module>
    from _k2 import DeterminizeWeightPushingType
ModuleNotFoundError: No module named '_k2'


### Install kaldifeat

In [6]:
! KALDIFEAT_MAKE_ARGS="-j" pip install --verbose kaldifeat

Using pip 23.0.1 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 482.4/482.4 kB 30.1 MB/s eta 0:00:00
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info
  writing /tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info/dependency_links.txt
  writing top-level names to /tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-rlg_oy0t/kaldifeat.egg-info/SOURCES.txt'
  Preparing meta

To check that kaldifeat was installed successfully, run

In [7]:
! python3 -c "import kaldifeat; print(kaldifeat.__version__)"

1.21


### Install icefall

icefall is a collection of Python scripts. All you need is just to
download its source code and set the `PYTHONPATH` environment variable.

In [8]:
! git clone https://github.com/k2-fsa/icefall

Cloning into 'icefall'...
remote: Enumerating objects: 11840, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 11840 (delta 90), reused 99 (delta 47), pack-reused 11658
Receiving objects: 100% (11840/11840), 13.73 MiB | 14.26 MiB/s, done.
Resolving deltas: 100% (8057/8057), done.


In [9]:
! pip install -q kaldialign sentencepiece>=0.1.96 lhotse

## Download pre-trained RNN-T Conformer model

To make the following steps easier, we decide to download the model
to `icefall/egs/tedlium3/ASR/tmp`

In [ ]:
! apt-get install -y -qq tree sox git-lfs

**CAUTION**: You have to run `sudo apt-get install git-lfs`
and `git lfs install` in order to download the pre-trained model.

In [ ]:
! cd icefall/egs/tedlium3/ASR && rm -rf tmp

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
  mkdir tmp && \
  cd tmp && \
  git lfs install && \
  git clone -v https://huggingface.co/luomingshuang/icefall_asr_tedlium3_transducer_stateless && \
  cd icefall_asr_tedlium3_transducer_stateless && \
  cd ../.. && \
  tree tmp

Updated git hooks.
Git LFS initialized.
Cloning into 'icefall_asr_tedlium3_transducer_stateless'...
POST git-upload-pack (165 bytes)
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 78 (delta 0), reused 4 (delta 0), pack-reused 74
Unpacking objects: 100% (78/78), 2.78 MiB | 2.50 MiB/s, done.
Filtering content: 100% (4/4), 2.21 GiB | 31.63 MiB/s, done.
tmp
└── icefall_asr_tedlium3_transducer_stateless
    ├── data
    │   └── lang_bpe_500
    │       └── bpe.model
    ├── exp
    │   ├── cpu_jit_epoch_19_avg_11_torch.1.7.1.pt
    │   ├── pretrained_average_14_to_29.pt
    │   └── pretrained_average_19_to_29.pt
    ├── log
    │   ├── beam_search
    │   │   ├── errs-dev-beam_4-epoch-29-avg-11-beam-4.txt
    │   │   ├── errs-dev-beam_4-epoch-29-avg-16-beam-4.txt
    │   │   ├── errs-test-beam_4-epoch-29-avg-11-beam-4.txt
    │   │   ├── errs-test-beam_4-epoch-29-avg-16-beam-4.txt
    │   │   ├─

In [ ]:
! soxi icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/*.wav


Input File     : 'icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W01.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:30.02 = 480256 samples ~ 2251.2 CDDA sectors
File Size      : 961k
Bit Rate       : 256k
Sample Encoding: 16-bit Signed Integer PCM


Input File     : 'icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W02.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:30.02 = 480256 samples ~ 2251.2 CDDA sectors
File Size      : 961k
Bit Rate       : 256k
Sample Encoding: 16-bit Signed Integer PCM


Input File     : 'icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W03.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:29.95 = 479232 samples ~ 2246.4 CDDA sectors
File Size      : 959k
Bit Rate       : 256k


## Greedy search

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
    PYTHONPATH=/content/icefall python3 ./transducer_stateless/pretrained.py \
      --method greedy_search \
      --max-sym-per-frame 1 \
      --checkpoint ./tmp/icefall_asr_tedlium3_transducer_stateless/exp/pretrained_average_19_to_29.pt \
      --bpe-model ./tmp/icefall_asr_tedlium3_transducer_stateless/data/lang_bpe_500/bpe.model \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W01.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W02.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W03.wav

2023-04-17 14:59:34,785 INFO [pretrained.py:249] {'sample_rate': 16000, 'feature_dim': 80, 'encoder_out_dim': 512, 'subsampling_factor': 4, 'attention_dim': 512, 'nhead': 8, 'dim_feedforward': 2048, 'num_encoder_layers': 12, 'vgg_frontend': False, 'env_info': {'k2-version': '1.23.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': '62e404dd3f3a811d73e424199b3408e309c06e1a', 'k2-git-date': 'Mon Jan 30 02:26:16 2023', 'lhotse-version': '1.13.0', 'torch-version': '1.13.1+cu117', 'torch-cuda-available': True, 'torch-cuda-version': '11.7', 'python-version': '3.9', 'icefall-git-branch': 'master', 'icefall-git-sha1': '8838fe0-clean', 'icefall-git-date': 'Mon Apr 17 09:47:25 2023', 'icefall-path': '/content/icefall', 'k2-path': '/usr/local/lib/python3.9/dist-packages/k2/__init__.py', 'lhotse-path': '/usr/local/lib/python3.9/dist-packages/lhotse/__init__.py', 'hostname': '3f5f2b213a05', 'IP address': '172.28.0.12'}, 'checkpoint': './tmp/icefall_asr_tedlium3_transducer_stateless

## Beam search

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
    PYTHONPATH=/content/icefall python3 ./transducer_stateless/pretrained.py \
      --method beam_search \
      --beam-size 4 \
      --checkpoint ./tmp/icefall_asr_tedlium3_transducer_stateless/exp/pretrained_average_19_to_29.pt \
      --bpe-model ./tmp/icefall_asr_tedlium3_transducer_stateless/data/lang_bpe_500/bpe.model \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W01.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W02.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W03.wav

2023-04-17 14:59:59,016 INFO [pretrained.py:249] {'sample_rate': 16000, 'feature_dim': 80, 'encoder_out_dim': 512, 'subsampling_factor': 4, 'attention_dim': 512, 'nhead': 8, 'dim_feedforward': 2048, 'num_encoder_layers': 12, 'vgg_frontend': False, 'env_info': {'k2-version': '1.23.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': '62e404dd3f3a811d73e424199b3408e309c06e1a', 'k2-git-date': 'Mon Jan 30 02:26:16 2023', 'lhotse-version': '1.13.0', 'torch-version': '1.13.1+cu117', 'torch-cuda-available': True, 'torch-cuda-version': '11.7', 'python-version': '3.9', 'icefall-git-branch': 'master', 'icefall-git-sha1': '8838fe0-clean', 'icefall-git-date': 'Mon Apr 17 09:47:25 2023', 'icefall-path': '/content/icefall', 'k2-path': '/usr/local/lib/python3.9/dist-packages/k2/__init__.py', 'lhotse-path': '/usr/local/lib/python3.9/dist-packages/lhotse/__init__.py', 'hostname': '3f5f2b213a05', 'IP address': '172.28.0.12'}, 'checkpoint': './tmp/icefall_asr_tedlium3_transducer_stateless

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
    PYTHONPATH=/content/icefall python3 ./transducer_stateless/pretrained.py \
      --method beam_search \
      --beam-size 4 \
      --checkpoint ./tmp/icefall_asr_tedlium3_transducer_stateless/exp/pretrained_average_19_to_29.pt \
      --bpe-model ./tmp/icefall_asr_tedlium3_transducer_stateless/data/lang_bpe_500/bpe.model \
      /content/84-121123-0002.flac \
      /content/fr1.wav

2023-04-17 15:17:53,303 INFO [pretrained.py:249] {'sample_rate': 16000, 'feature_dim': 80, 'encoder_out_dim': 512, 'subsampling_factor': 4, 'attention_dim': 512, 'nhead': 8, 'dim_feedforward': 2048, 'num_encoder_layers': 12, 'vgg_frontend': False, 'env_info': {'k2-version': '1.23.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': '62e404dd3f3a811d73e424199b3408e309c06e1a', 'k2-git-date': 'Mon Jan 30 02:26:16 2023', 'lhotse-version': '1.13.0', 'torch-version': '1.13.1+cu117', 'torch-cuda-available': True, 'torch-cuda-version': '11.7', 'python-version': '3.9', 'icefall-git-branch': 'master', 'icefall-git-sha1': '8838fe0-clean', 'icefall-git-date': 'Mon Apr 17 09:47:25 2023', 'icefall-path': '/content/icefall', 'k2-path': '/usr/local/lib/python3.9/dist-packages/k2/__init__.py', 'lhotse-path': '/usr/local/lib/python3.9/dist-packages/lhotse/__init__.py', 'hostname': '3f5f2b213a05', 'IP address': '172.28.0.12'}, 'checkpoint': './tmp/icefall_asr_tedlium3_transducer_stateless

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
  export CUDA_VISIBLE_DEVICES="0,1,2,3" && \
  python ./transducer_stateless/train.py  && \
  --world-size 4 && \
  --num-epochs 30 \
  --start-epoch 0 \
  --exp-dir transducer_stateless/exp \
  --max-duration 300

Traceback (most recent call last):
  File "/content/icefall/egs/tedlium3/ASR/./transducer_stateless/train.py", line 44, in <module>
    from asr_datamodule import TedLiumAsrDataModule
  File "/content/icefall/egs/tedlium3/ASR/transducer_stateless/asr_datamodule.py", line 37, in <module>
    from icefall.utils import str2bool
ModuleNotFoundError: No module named 'icefall'


## Train

In [10]:
! cd icefall && \
  ! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00


In [11]:

import os

!pwd
! echo $PYTHONPATH


os.environ['PYTHONPATH'] += ":/content/icefall"

! echo $PYTHONPATH

/content
/env/python
/env/python:/content/icefall


In [13]:
pip list  | grep torch

torch                         1.13.1
torchaudio                    2.0.1+cu118
torchdata                     0.6.0
torchsummary                  1.5.1
torchtext                     0.15.1
torchvision                   0.15.1+cu118


In [14]:
pip install torchaudio==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118


In [15]:
! cd icefall/egs/tedlium3/ASR && \
  ./prepare.sh

2023-04-29 12:20:37 (prepare.sh:50:main) dl_dir: /content/icefall/egs/tedlium3/ASR/download
2023-04-29 12:20:37 (prepare.sh:53:main) Stage 0: Download data
--2023-04-29 13:39:57--  http://kaldi-asr.org/models/5/4gram_small.arpa.gz
Resolving kaldi-asr.org (kaldi-asr.org)... 46.101.158.64
Connecting to kaldi-asr.org (kaldi-asr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18358716 (18M) [application/x-gzip]
Saving to: ‘/content/icefall/egs/tedlium3/ASR/download/lm/4gram_small.arpa.gz’

4gram_small.arpa.gz 100%[===================>]  17.51M  6.75MB/s    in 2.6s    

2023-04-29 13:40:00 (6.75 MB/s) - ‘/content/icefall/egs/tedlium3/ASR/download/lm/4gram_small.arpa.gz’ saved [18358716/18358716]

--2023-04-29 13:40:00--  http://kaldi-asr.org/models/5/4gram_big.arpa.gz
Resolving kaldi-asr.org (kaldi-asr.org)... 46.101.158.64
Connecting to kaldi-asr.org (kaldi-asr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Le